In [1]:
from pathlib import Path
import requests

In [2]:
DATA_PATH = Path("data")
PATH = DATA_PATH/"mnist"

#PATH.mkdir(parents = True, exist_ok = True)
URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

In [3]:
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

In [4]:
print(x_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
import matplotlib.pyplot as plt
import numpy as np

plt.imshow(x_train[0].reshape((28,28)),cmap = 'gray')
plt.show()

<Figure size 640x480 with 1 Axes>

In [6]:
type(x_train) # x_train is the ndarray form

numpy.ndarray

In [7]:
import torch

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)
n, c = x_train.shape

In [8]:
print("n:\n",n)
print("c:\n",c)

n:
 50000
c:
 784


In [9]:
import math

#  These are just regular tensors, with one very special addition: we tell PyTorch that they require a gradient. This causes PyTorch to record all of the operations done on the tensor, so that it can calculate the gradient during back-propagation automatically!
weights = torch.randn(784,10)/math.sqrt(784)
# tell the program to give access to automatically compute the weights gradient
weights.requires_grad_()
bias = torch.zeros(10, requires_grad = True)
#print(bias)

In [10]:
def log_softmax(x):
    return x - x.exp().sum(1).log().unsqueeze(-1)
def model(xb):
    # @represent the dot of matrices 
    return log_softmax(xb @ weights + bias)

In [11]:
a = torch.rand(3,5)
print(a)

tensor([[0.8944, 0.6385, 0.7916, 0.3534, 0.7680],
        [0.2882, 0.0538, 0.6395, 0.2962, 0.2814],
        [0.3495, 0.8762, 0.7002, 0.3274, 0.8281]])


In [12]:
log_softmax(a)

tensor([[-1.4207, -1.6766, -1.5234, -1.9617, -1.5471],
        [-1.6512, -1.8856, -1.2999, -1.6432, -1.6580],
        [-1.9029, -1.3762, -1.5522, -1.9250, -1.4243]])

In [1]:
print(x_train)

NameError: name 'x_train' is not defined

In [ ]:
bs = 64 # batch size
xb = x_train[0:bs] # a mini-batch from x
preds = model(xb) # prediction
print(preds.shape)

In [ ]:
def nll(input, target):
    return -input[range(target.shape[0]), target].mean()
loss_func = nll    

In [ ]:
yb = y_train[0:bs]
print(loss_func(preds, yb))

In [ ]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim = 1)
    return (preds == yb).float().mean()
print(accuracy(preds, yb))

In [ ]:
"""
select a mini-batch of data (of size bs)
use the model to make predictions
calculate the loss
loss.backward() updates the gradients of the model, in this case, weights and bias.
"""

In [ ]:
print((n-1)//bs + 1)
print(weights.shape)
print(bias)

In [ ]:
from IPython.core.debugger import set_trace

lr = 0.5 # learning rate
epochs = 2 # how many epochs to train for

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        # set_trace()
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()
            
print(weights)
print(bias)

In [ ]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))
# it means that every row has the right value

In [ ]:
import torch.nn.functional as F

loss_func = F.cross_entropy
def model(xb):
    return xb @ weight + bias